In [191]:
!pip install langdetect==1.0.9
!pip install googletrans==3.1.0a0
!pip install iso639==0.1.4
!pip install yake==0.4.8
!pip install lemminflect==0.2.3
!python -m spacy download en_core_web_md

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
2022-11-14 07:49:42.112145: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 42.8 MB 2.5 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


In [192]:
import requests
from requests.packages.urllib3.exceptions import InsecureRequestWarning 

from bs4 import BeautifulSoup
from bs4 import BeautifulSoup
from bs4.element import Comment

import numpy as np
import re
import string

# Disable displaying SSL verification warnings
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

HEADERS = ({'User-Agent':
            'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36',
            'Accept-Language': 'en-US, en;q=0.5'})

In [195]:
def tag_visible(element):
  # blacklist = ['style', 'label', '[document]', 'embed', 'img', 'object',
  #             'noscript', 'header', 'html', 'iframe', 'audio', 'picture',
  #             'meta', 'title', 'aside', 'footer', 'svg', 'base', 'figure',
  #             'form', 'nav', 'head', 'link', 'button', 'source', 'canvas',
  #             'br', 'input', 'script', 'wbr', 'video', 'param', 'hr', 'li', 'ol', 'ul', 'table', 'th', 'tr', 'td', 'a', 'button', 'span']

  required = ['title', 'h1', 'h2','h3', 'h4', 'h5', 'h6', 'strong', 'bold', 'p']
              
  if element.parent.name not in required:
      return False
  if isinstance(element, Comment):
      return False
  return True

In [196]:
def getTextFromURL(url):
  try:
      page = requests.get(url, headers=HEADERS)          #to extract page from website
      html_code = page.content                           #to extract html code from page

      soup = BeautifulSoup(html_code, 'html.parser')     #Parse html code
      text = soup.findAll(text=True)                     #find all text

      text_from_html = ''

      visible_texts = filter(tag_visible, text)  
      text_from_html = " ".join(t.strip() for t in visible_texts)

      text_from_html = text_from_html.strip()

      text_from_html = re.sub('\n', ' ', text_from_html)
      res = re.sub(' +', ' ', text_from_html)

      return res

  except Exception as e:
      print(e)

In [197]:
import re
from langdetect import detect
from googletrans import Translator


def preProcessForTranslation(text):
    # removing english words
    txt_list = list(text.split(" "))
    txt = list(filter(lambda ele: re.search("[a-zA-Z\s]+", ele) is None, txt_list))
    text = ' '.join([str(elem) for elem in txt])

    return str(text.strip())



def detect_and_translate(text, target_lang):
    
    result_lang = detect(text)
    # print(result_lang)
    
    if result_lang == target_lang:
        return text 
    
    else:
        translatedText = ""
        translator= Translator()
        text = preProcessForTranslation(text)

        txt_list = list(text.split(" "))
        chunks = [txt_list[x:x+100] for x in range(0, len(txt_list), 100)]    # limit is of 5000 characters 
        for i in chunks:
            text = ' '.join([str(elem) for elem in i])
            translation = translator.translate(text, src=result_lang, dest=target_lang)
            translatedText = translatedText + ' ' + translation.text
        
        return translatedText

In [198]:
import yake
import string
import nltk
# nltk.download('stopwords')
# nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import spacy
import lemminflect
from itertools import combinations

nlp = spacy.load("en_core_web_md")



def pre_process(titles):
    """
    Pre-processes titles by removing stopwords and lemmatizing text.
    :param titles: list of strings, contains target titles,.
    :return: preprocessed_title_docs, list containing pre-processed titles.
    """

    # Preprocess all the titles
    title_docs = [nlp(x) for x in titles]
    preprocessed_title_docs = []
    lemmatized_tokens = []
    for title_doc in title_docs:
        for token in title_doc:
            if not token.is_stop:
                lemmatized_tokens.append(token.lemma_)
        preprocessed_title_docs.append(" ".join(lemmatized_tokens))
        del lemmatized_tokens[
            :
            ]  # empty the lemmatized tokens list as the code moves onto a new title

    return preprocessed_title_docs



def similarity_filter(titles):
    """
    Recursively check if titles pass a similarity filter.
    :param titles: list of strings, contains titles.
    If the function finds titles that fail the similarity test, the above param will be the function output.
    :return: this method upon itself unless there are no similar titles; in that case the feed that was passed
    in is returned.
    """

    # Preprocess titles
    preprocessed_title_docs = pre_process(titles)
    # print(preprocessed_title_docs)
    # Remove similar titles
    all_summary_pairs = list(combinations(preprocessed_title_docs, 2))
    similar_titles = []
    for pair in all_summary_pairs:
        title1 = nlp(pair[0])
        title2 = nlp(pair[1])
        similarity = title1.similarity(title2)
        if similarity > 0.8:
            similar_titles.append(pair)

    titles_to_remove = []
    for a_title in similar_titles:
        # Get the index of the first title in the pair
        index_for_removal = preprocessed_title_docs.index(a_title[1])
        titles_to_remove.append(index_for_removal)

    # Get indices of similar titles and remove them
    similar_title_counts = set(titles_to_remove)
    similar_titles = [
        x[1] for x in enumerate(titles) if x[0] in similar_title_counts
    ]

    # Exit the recursion if there are no longer any similar titles
    if len(similar_title_counts) == 0:
        return titles

    # Continue the recursion if there are still titles to remove
    else:
        # Remove similar titles from the next input
        for title in similar_titles:
            idx = titles.index(title)
            titles.pop(idx)
            
        return similarity_filter(titles)




def getDomainTopics(text):
    
    # PREPROCESSING
    # 1. Remove punctuation
    # 2. Remove digits
    # 3. Convert to lowercase
    # 4. Remove stop words
    # 5. Lemmatization


    def remove_punctuation(text):
        punctuationfree="".join([i for i in text if i not in string.punctuation])
        return punctuationfree

    text=remove_punctuation(text)
    text = text.lower()
    text = ''.join([i for i in text if not i.isdigit()])
    text=" ".join(text.split())


    # filtered_list = []
    # stop_words = nltk.corpus.stopwords.words('english')
    # words = word_tokenize(text)
    # for w in words:
    #     if w.lower() not in stop_words:
    #         filtered_list.append(w)
            
    # text = " ".join(filtered_list)


    nlp = spacy.load('en_core_web_md')
    doc = nlp(text)
    lemmatised=[]

    for item in doc:
        item = item._.lemma()
        lemmatised.append(item)

    text=" ".join(lemmatised)


    # YAKE
    # optimise parameters to get best results
    language = "en"
    max_ngram_size = 2
    deduplication_thresold = 0.9
    deduplication_algo = 'seqm'
    windowSize = 1
    numOfKeywords = 10

    kw_extractor = yake.KeywordExtractor(lan=language, 
                                        n=max_ngram_size, 
                                        dedupLim=deduplication_thresold, 
                                        dedupFunc=deduplication_algo, 
                                        windowsSize=windowSize, 
                                        top=numOfKeywords)
                                                

    keywords = kw_extractor.extract_keywords(text)

    topics = []

    for kw, v in keywords:
      words = kw.split()
      keyw = " ".join(sorted(set(words), key=words.index))
      topics.append(keyw)
        #   print("Keyphrase: ",keyw, ": score", 100-v)
    filtered_topics = similarity_filter(topics)

    domain = keywords[0][0]
    topics = filtered_topics

    return domain

In [199]:
import iso639
from langdetect import detect

results = {}
links = ['https://medium.com/analytics-vidhya/topic-modelling-using-lda-aa11ec9bec13', 'https://timesofindia.indiatimes.com/', 'https://www.bbc.com/korean/features-63100903', 'https://www.amazon.in/Amazon-Brand-Solimo-Single-Recliner/dp/B0862RSLNX/ref=sr_1_7?qid=1664600663&refinements=p_36%3A3444814031&rnid=3444809031&s=kitchen&sr=1-7', 'https://www.aajtak.in/technology/tech-news/story/pm-narendra-modi-launch-5g-in-india-5g-speed-5g-data-jio-airtel-vi-ttec-1547502-2022-10-01', 'https://www.mk.ru/politics/2022/09/30/putin-raskryl-ne-vse-karty-glavnye-syurprizy-kremlya-vperedi.html?from=main_omk', 'https://gujarati.news18.com/news/tech/imc-2022-electricity-theft-will-be-curbed-dp-1258916.html', 'https://www.flipkart.com/google-pixel-6a-charcoal-128-gb/p/itme5ae89135d44e?pid=MOBGFKX5YUXD74Z3&lid=LSTMOBGFKX5YUXD74Z3MXA2OB&marketplace=FLIPKART&store=tyy%2F4io&srno=b_1_1&otracker=clp_bannerads_1_24.bannerAdCard.BANNERADS_A_mobile-phones-store_WPMQ7E9TZNG3&fm=organic&iid=9cf25f5f-2a59-47dd-b846-1091bb3d8f2a.MOBGFKX5YUXD74Z3.SEARCH&ppt=clp&ppn=mobile-phones-store&ssid=ehewriyw8w0000001667719220727', 'https://byjus.com/jee-advanced/#jee-advanced-counselling']

for i in links:
  rawOriginalText = getTextFromURL(i)

  rawText=""
  original_lang = iso639.to_name(detect(rawOriginalText))
  if original_lang != 'English':
      rawText = detect_and_translate(rawOriginalText, target_lang='en')
  else:
      rawText = rawOriginalText
  domain = getDomainTopics(rawText)

  results[i] = domain

print(results)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:58: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.


{'https://medium.com/analytics-vidhya/topic-modelling-using-lda-aa11ec9bec13': 'topic modelling', 'https://timesofindia.indiatimes.com/': 'bomma blockbuster', 'https://www.bbc.com/korean/features-63100903': 'sea level', 'https://www.amazon.in/Amazon-Brand-Solimo-Single-Recliner/dp/B0862RSLNX/ref=sr_1_7?qid=1664600663&refinements=p_36%3A3444814031&rnid=3444809031&s=kitchen&sr=1-7': 'solimo biela', 'https://www.aajtak.in/technology/tech-news/story/pm-narendra-modi-launch-5g-in-india-5g-speed-5g-data-jio-airtel-vi-ttec-1547502-2022-10-01': 'telecom service', 'https://www.mk.ru/politics/2022/09/30/putin-raskryl-ne-vse-karty-glavnye-syurprizy-kremlya-vperedi.html?from=main_omk': 'russian federation', 'https://gujarati.news18.com/news/tech/imc-2022-electricity-theft-will-be-curbed-dp-1258916.html': 'curb power', 'https://www.flipkart.com/google-pixel-6a-charcoal-128-gb/p/itme5ae89135d44e?pid=MOBGFKX5YUXD74Z3&lid=LSTMOBGFKX5YUXD74Z3MXA2OB&marketplace=FLIPKART&store=tyy%2F4io&srno=b_1_1&otrack